In [1]:
import os
import pandas as pd
import arcpy
from arcpy import env
from arcpy.management import *
from arcpy.sa import *
from arcpy.da import *
from arcpy.conversion import *
from arcpy.analysis import *
import numpy as np

In [2]:
# tool function
# 创建渔网
def create_net(output_file,extent_file,cell_size):
    """参数说明：
        output_file: 输出文件
        extent_file: 输入范围文件
        cell_size: 生成的cell大小
    """
    proj = arcpy.Describe(extent_file).spatialReference
    extent_x_min = arcpy.Describe(extent_file).extent.XMin
    extent_y_min = arcpy.Describe(extent_file).extent.YMin
    extent_x_max = arcpy.Describe(extent_file).extent.XMax
    extent_y_max = arcpy.Describe(extent_file).extent.YMax
    with arcpy.EnvManager(outputCoordinateSystem = proj):
        arcpy.management.CreateFishnet(out_feature_class=output_file,origin_coord=f"{extent_x_min} {extent_y_min}",y_axis_coord=f"{extent_x_min} {extent_y_min+10}",cell_width=cell_size,cell_height=cell_size,number_rows=None,number_columns=None,corner_coord=f"{extent_x_max} {extent_y_max}",labels="NO_LABELS",template="#",geometry_type="POLYGON")
# 面转点
def polygon_point(in_feature, out_feature):
    """参数说明：
        in_feature: 输入面
        out_feature: 输出点
    """
    proj = arcpy.Describe(in_feature).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=proj):
        arcpy.management.FeatureToPoint(in_features=in_feature,out_feature_class=out_feature,point_location="INSIDE")

# 筛选点

def select_point(input_polygon,input_point,output_point):
    """参数说明：
        input_polygon: 选择面
        input_point: 待选择点
        output_point: 选择点
    """
    polygon_layer = "polygonLayer"
    point_layer = "pointLayer"
    arcpy.MakeFeatureLayer_management(input_polygon, polygon_layer)
    arcpy.MakeFeatureLayer_management(input_point, point_layer)
    arcpy.SelectLayerByLocation_management(in_layer=point_layer, overlap_type="INTERSECT", select_features=polygon_layer,search_distance=0,selection_type='NEW_SELECTION',invert_spatial_relationship="NOT_INVERT")
    # 导出选择的要素
    arcpy.CopyFeatures_management(point_layer, output_point)

# 添加字段
def add_field(input_table,field_name,field_type='TEXT'):
    """参数说明：
        input_table: 输入数据表
        field_name: 字段名
        field_type: 字段类型"""
    arcpy.AddField_management(input_table,field_name,field_type)
# 删除要素
def delete_feature(input_feature):
    arcpy.Delete_management(input_feature)
# 空间连接
def perform_spatial_join(target_layer_path, join_layer_path, output_layer_path, field_mapping_dict):
    """参数说明：
        target_layer_path: 目标图层路径 (点图层)
        join_layer_path: 连接图层路径 (包含属性的面图层)
        output_layer_path: 输出图层路径
        field_mapping_dict: 字段映射字典 {目标字段:源字段}
    """
    # 创建空的 FieldMappings 对象
    field_mappings = arcpy.FieldMappings()

    # 只添加需要的字段映射
    for target_field, source_field in field_mapping_dict.items():
        # 创建字段映射对象
        field_map = arcpy.FieldMap()
        
        # 添加源字段（从连接图层）
        field_map.addInputField(join_layer_path, source_field)
        
        # 设置输出字段属性
        output_field = field_map.outputField
        output_field.name = target_field
        output_field.aliasName = target_field
        field_map.outputField = output_field
        
        # 添加到field_mappings
        field_mappings.addFieldMap(field_map)

    # 执行空间连接操作
    arcpy.analysis.SpatialJoin(
        target_layer_path, 
        join_layer_path, 
        output_layer_path,
        "JOIN_ONE_TO_ONE", 
        "KEEP_ALL", 
        field_mappings,
        match_option="INTERSECT"
    )


# 点采样
def sample_point(point_,raster_,out_name):
    """根据栅格采样点,输出为表格"""
    Sample(raster_,point_,out_name,"NEAREST", "OBJECTID", "CURRENT_SLICE", None, '', None, None, "ROW_WISE", "TABLE")
    return None

# 导出CSV
def export_csv(table_,out_path,out_name):
    """参数说明：
        table_: 输入数据表
        out_path: 输出路径
        out_name: 输出表名"""
    TableToTable(table_,out_path,out_name)
    return None

# 要素转点
def feature_to_point(input_layer,output_layer):
    """参数说明："""
    proj = arcpy.Describe(input_layer).spatialReference
    with arcpy.EnvManager(outputCoordinateSystem=proj):
        arcpy.management.FeatureToPoint(input_layer, output_layer, "INSIDE")

# 删除多余字段
def delete_extra_fields(input_feature_class, fields_list):
    """参数说明："""
    fields = arcpy.ListFields(input_feature_class)
    for field in fields:
        name = field.name
        type = field.type.upper()
        # 如果字段名不在删选列表中且类型不为OID和Geometry便删除这个字段
        if name not in fields_list and type not in ['OID','GEOMETRY']:
            arcpy.DeleteField_management(input_feature_class, [name])

In [3]:
# 设置工作空间
env.workspace = r'F:\cache_data\shp_file\sb'
env.overwriteOutput=True

In [4]:
def create_point(input_layer):
    """生成虚拟点"""
    # 定义文件名称
    fish_net_name = "gl_grid"
    net_to_point_name = "gl_inside_point"
    select_point_name = "select_point"
    result_point_name = "join_point"
    # 渔网构建
    create_net(fish_net_name,input_layer,300)
    # 要素转点
    polygon_point(fish_net_name,net_to_point_name)
    # 按位置选择
    select_point(input_layer,net_to_point_name,select_point_name)
    # 添加字段
    field_list = ['TL','YL','TS','TZ']
    for one_field in field_list:
        # 添加字段
        add_field(input_table=select_point_name,field_name=one_field,field_type='TEXT')
    # 空间连接赋予地类属性
    # 定义目标要素图层和连接要素图层的路径
    fields_mapping = {
        "TL": "土类",
        "YL": "亚类",
        "TS": "土属",
        "TZ": "土种"
    }
    # 空间连接
    perform_spatial_join(select_point_name,input_layer,result_point_name,fields_mapping)
    # 删除多余数据
    for one_feature in [fish_net_name,net_to_point_name,select_point_name]:
        delete_feature(one_feature)

In [5]:
# 生成渔网点
create_point(input_layer="sb_ep_polygon")

In [6]:
# 生成面内部点
feature_to_point(input_layer="sb_ep_polygon",output_layer="sb_tl_dh_point")

In [7]:
# 内部点字段整理
input_feature = r"F:\cache_data\shp_file\sb\sb_tl_dh_point.shp"
# 定义字段映射关系：新字段名:原字段名
field_mapping = {
    'TL': '土类',
    'YL': '亚类',
    'TS': '土属',
    'TZ': '土种'
}

# 添加字段并复制值
for new_field, source_field in field_mapping.items():
    # 添加新字段
    add_field(input_table=input_feature, field_name=new_field, field_type='TEXT')
    # 从原有字段复制值到新字段
    arcpy.CalculateField_management(input_feature, new_field, f"!{source_field}!", "PYTHON3")

# 只保留新添加的字段
delete_extra_fields(input_feature, list(field_mapping.keys()))

In [8]:
# 渔网点字段整理
# 添加字段
input_feature = "join_point"
# input_feature = r"sb_tl_dh_point.shp"
field_list = ['TL','YL','TS','TZ']
# 删除多余字段
delete_extra_fields(input_feature, field_list)

In [9]:
# 为两个点位数据增加label
feature_1 = "join_point"
feature_2 = "sb_tl_dh_point"
for one_feature in [[feature_1,'fish_net'],[feature_2,'inner']]:
    features = one_feature[0]
    labels = one_feature[1]
    field_name = "label"
    # 添加字段
    add_field(input_table=features,field_name="label",field_type='TEXT')
    # 注意：这里使用了Python表达式
    arcpy.CalculateField_management(features, field_name, f"'{labels}'", "PYTHON3")


In [10]:
# 合并点位数据
feature_1 = "join_point"
feature_2 = "sb_tl_dh_point"
output_feature = "sb_filter_result_point"
# 合并
arcpy.management.Merge([feature_1,feature_2],output_feature)
# 删除过渡文件
for one_feature in [feature_1,feature_2]:
    delete_feature(one_feature)

In [ ]:
# 添加经纬度字段并计算经纬度

# 输入和输出文件
input_feature_class = "sb_filter_result_point"  # 请替换为您的点文件的名称

# 添加字段
arcpy.AddField_management(input_feature_class, "LON", "DOUBLE")
arcpy.AddField_management(input_feature_class, "LAT", "DOUBLE")

# 使用CalculateField_management计算经纬度
# 假设您的点文件有一个名为"SHAPE"的字段，它包含了点的几何信息
# arcpy.CalculateField_management(input_feature_class, "LON", "!SHAPE.firstPoint.X!", "PYTHON3")
# arcpy.CalculateField_management(input_feature_class, "LAT", "!SHAPE.firstPoint.Y!", "PYTHON3")
arcpy.management.CalculateGeometryAttributes(input_feature_class, [['LON','POINT_X'], ['LAT','POINT_Y']],coordinate_format='DD')
# print("字段添加和计算完成！")